# Scoring of variations

There are three different cases:

1. Simple SNPs
2. Indels on the query
3. SNPs at a distance less than  K


In [1]:
from IBSpy import KmerGWASDBBuilder, KmerGWASDB, FastaChunkReader 
from pyfaidx import Fasta

In [2]:
kmerdb  = KmerGWASDB(31)
kmerdb.load_from_fasta("../tests/data/test4B.jagger.fa")
builder = KmerGWASDBBuilder(31)

In [3]:
fasta_iter = FastaChunkReader("../tests/data/test4B.stanley.rev.fa", kmer_size = 31)
sequence_sta = list(fasta_iter )[0]
fasta_iter = FastaChunkReader("../tests/data/test4B.jagger.fa", kmer_size = 31)
sequence_jag = list(fasta_iter )[0]

## Simple case (SNP)

A single SNP is recorded as a single variation, a kmer_distance of 1 and the observed kmers is $total_kmers - k$. In this example, 33 kmers, with a SNP in the centre only display 2 observed kmers. 

In [4]:
jag1  = sequence_jag["seq"][4516:4579].lower() 
sta1  = sequence_sta["seq"][4518:4581].lower()
position = 31
jag1 = jag1[:position] + jag1[position].upper() + jag1[position+1:]
sta1 = sta1[:position] + sta1[position].upper() + sta1[position+1:]
query = list(builder.sequence_to_kmers(sta1, convert=True))
print(jag1)
print(sta1)
i = 0
count = 0 
for q in query:
    q_str = builder.kmer_to_string(q)
    q_str = " "*i + q_str
    if(q not in kmerdb):
        q_str = q_str.lower()
        q_str = q_str[:position] + q_str[position].upper() + q_str[position+1:]
    print(q_str)
    i += 1

stats = kmerdb.kmers_stats_from_sequence(query)
stats.seqname = "jag1"
stats.start   = 0
stats.end     = len(query) - 1
print(stats.header())
print(stats.csv())

cctcgcctccgcgacctggagcgattgctggTcgtcggctatgtgcccgcgcagcgccgtgcc
cctcgcctccgcgacctggagcgattgctggGcgtcggctatgtgcccgcgcagcgccgtgcc
CCTCGCCTCCGCGACCTGGAGCGATTGCTGG
 ctcgcctccgcgacctggagcgattgctggG
  tcgcctccgcgacctggagcgattgctggGc
   cgcctccgcgacctggagcgattgctggGcg
    gcctccgcgacctggagcgattgctggGcgt
     cctccgcgacctggagcgattgctggGcgtc
      ctccgcgacctggagcgattgctggGcgtcg
       tccgcgacctggagcgattgctggGcgtcgg
        ccgcgacctggagcgattgctggGcgtcggc
         cgcgacctggagcgattgctggGcgtcggct
          gcgacctggagcgattgctggGcgtcggcta
           cgacctggagcgattgctggGcgtcggctat
            gacctggagcgattgctggGcgtcggctatg
             acctggagcgattgctggGcgtcggctatgt
              cctggagcgattgctggGcgtcggctatgtg
               ctggagcgattgctggGcgtcggctatgtgc
                tggagcgattgctggGcgtcggctatgtgcc
                 ggagcgattgctggGcgtcggctatgtgccc
                  gagcgattgctggGcgtcggctatgtgcccg
                   agcgattgctggGcgtcggctatgtgcccgc
                    gcgattgctggGcgtcggctat

## Indels

Indels produce a single variation, a kmer distance of the size of the deletion and the total number of kmers is reduced by k plus the deleiton size


In [5]:
jag1  = sequence_jag["seq"][1948:2005]
sta1  = sequence_sta["seq"][1948:2007]
position = 31
jag1 = jag1[:position] + "--" + jag1[position:]
query = list(builder.sequence_to_kmers(sta1, convert=True))
print(jag1)
print(sta1)
i = 0
count = 0 
for q in query:
    q_str = builder.kmer_to_string(q)
    q_str = " "*i + q_str
    if(q not in kmerdb):
        q_str = q_str.lower()
        #q_str = q_str[:position] + q_str[position].upper() + q_str[position+1:]
    print(q_str)
    i += 1

stats = kmerdb.kmers_stats_from_sequence(query)
stats.seqname = "jag1"
stats.start   = 0
stats.end     = len(query) - 1
print(stats.header())
print(stats.csv())

TTGCATTAATTCATTTAGATCTTTTGACACA--GGCAGCAGTTCCCAAGATTCTTTTGG
TTGCATTAATTCATTTAGATCTTTTGACACACAGGCAGCAGTTCCCAAGATTCTTTTGG
TTGCATTAATTCATTTAGATCTTTTGACACA
 tgcattaattcatttagatcttttgacacac
  gcattaattcatttagatcttttgacacaca
   cattaattcatttagatcttttgacacacag
    attaattcatttagatcttttgacacacagg
     ttaattcatttagatcttttgacacacaggc
      taattcatttagatcttttgacacacaggca
       aattcatttagatcttttgacacacaggcag
        attcatttagatcttttgacacacaggcagc
         ttcatttagatcttttgacacacaggcagca
          tcatttagatcttttgacacacaggcagcag
           catttagatcttttgacacacaggcagcagt
            atttagatcttttgacacacaggcagcagtt
             tttagatcttttgacacacaggcagcagttc
              ttagatcttttgacacacaggcagcagttcc
               tagatcttttgacacacaggcagcagttccc
                agatcttttgacacacaggcagcagttccca
                 gatcttttgacacacaggcagcagttcccaa
                  atcttttgacacacaggcagcagttcccaag
                   tcttttgacacacaggcagcagttcccaaga
                    cttttgacacacaggcagcagttcccaaga

## SNPs closer at a distance less than K 

When SNPs are close by, they are registered as a single variation. However, the kmer_distance of the event is longer, as all the kmers between the SNPs are missing. In this case, the kmer_distance is calculated by the distance between the first and the last SNP plus one. If there were more SNPs in between, the distance would still be the same. This hides regions with a different level of conservation, but we can't resolve it just with the kmer database. We need the connections between kmers to resolve the path.   

In [6]:
jag1  = sequence_jag["seq"][3348:3440].lower()
sta1  = sequence_sta["seq"][3350:3442].lower()
position = 31
jag1 = jag1[:position] + jag1[position].upper() + jag1[position+1:]
sta1 = sta1[:position] + sta1[position].upper() + sta1[position+1:]
position2 = 31 + 29
jag1 = jag1[:position2] + jag1[position2].upper() + jag1[position2+1:]
sta1 = sta1[:position2] + sta1[position2].upper() + sta1[position2+1:]
query = list(builder.sequence_to_kmers(sta1, convert=True))
print(jag1)
print(sta1)
i = 0
count = 0 
for q in query:
    q_str = builder.kmer_to_string(q)
    q_str = " "*i + q_str
    if(q not in kmerdb):
        q_str = q_str.lower()
        q_str = q_str[:position] + q_str[position].upper() + q_str[position+1:]
        if i > 29:
            q_str = q_str[:position2] + q_str[position2].upper() + q_str[position2+1:]
    print(q_str)
    i += 1

stats = kmerdb.kmers_stats_from_sequence(query)
stats.seqname = "jag1"
stats.start   = 0
stats.end     = len(query) - 1
print(stats.header())
print(stats.csv())

acaaaaggcgagtaaaagagttctctgttcaCcggctacgagcagcgtagacaaaggaagTggcggcgataacagtcgtggtagggcacagc
acaaaaggcgagtaaaagagttctctgttcaAcggctacgagcagcgtagacaaaggaagCggcggcgataacagtcgtggtagggcacagc
ACAAAAGGCGAGTAAAAGAGTTCTCTGTTCA
 caaaaggcgagtaaaagagttctctgttcaA
  aaaaggcgagtaaaagagttctctgttcaAc
   aaaggcgagtaaaagagttctctgttcaAcg
    aaggcgagtaaaagagttctctgttcaAcgg
     aggcgagtaaaagagttctctgttcaAcggc
      ggcgagtaaaagagttctctgttcaAcggct
       gcgagtaaaagagttctctgttcaAcggcta
        cgagtaaaagagttctctgttcaAcggctac
         gagtaaaagagttctctgttcaAcggctacg
          agtaaaagagttctctgttcaAcggctacga
           gtaaaagagttctctgttcaAcggctacgag
            taaaagagttctctgttcaAcggctacgagc
             aaaagagttctctgttcaAcggctacgagca
              aaagagttctctgttcaAcggctacgagcag
               aagagttctctgttcaAcggctacgagcagc
                agagttctctgttcaAcggctacgagcagcg
                 gagttctctgttcaAcggctacgagcagcgt
                  agttctctgttcaAcggctacgagcagcgta
                   gttctctgttcaAcgg